In [28]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import csv
 
class TwitterClient(object):
    def __init__(self):
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'XfcbXSUkdUBxbKZFRI0iwzFta'
        consumer_secret = 'JM9FM9kYfpiGkCpfw0vaeUjNoY9VKO1lCBHgFx3gJE83DQ3DB0'
        access_token = '1031510125784121344-PyoDNc5LJVhpI4q0yiOeG7WprrnAQJ'
        access_token_secret = 'mcvN9YOdGZDdPt4DwO8ZUo5AA8iAXx0D4DHTOPdxURQ1Q'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)   
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

In [29]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'humalog', count = 200)
 
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    #print("Neutral tweets percentage: {} %".format(100*len(tweets - ntweets - ptweets)/len(tweets)))
 
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
 
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
 
if __name__ == "__main__":
    # calling main function
    main()

Positive tweets percentage: 18.0 %
Negative tweets percentage: 14.0 %


Positive tweets:
RT @pudwellphoto: Just tried to fill my Humalog insulin order for 3 month supply and it was $2700.  Thanks @LillyPad for getting richer whi…
RT @MelindaMWedding: I needed a reminder of why I speak so loudly about #insulin4all so I pulled my act of gratitude for decent insurance.…
OPERATION ANTI-CORPSE: DAY ONE (Part 1/3)
Wgt: 279.2
Bld Sugar: 304
BP: 131/78
PR: 77
B'fast (7am): 2eggs scram + h… https://t.co/dURQgzRLwa
HUMALOG MIX 25 100IU/3ML CARTRIDGE

Humalog Mix 25 100IU/ml Cartridg..For more info visit...https://t.co/n25DbzMidE https://t.co/JlNaafDMs6
The onset of action of rapid-acting Humalog is within 10-15 minutes.
RT @brendanphyland: @_chuck956 docs prescribe Novolog Humalog, Apidra and Fiasp. All of these insulins keep people alive but have massive l…
RT @robscatswayze: Meow Hey @LillyDiabetes so how much is Humalog? Because if you “cared” about diabetics it would be cheap. Signed, thankf